In [4]:
%load_ext autoreload
%autoreload 2
%aimport -decode_fish.engine.place_psfs

In [5]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

from decode_fish.imports import *
from decode_fish.funcs.file_io import *
from decode_fish.funcs.emitter_io import *
from decode_fish.funcs.utils import *
from decode_fish.funcs.dataset import *
from decode_fish.funcs.plotting import *
from decode_fish.engine.noise import estimate_noise_scale
from decode_fish.engine.microscope import add_pos_noise
import shutil
from decode_fish.funcs.visualization import *
from decode_fish.funcs.matching import *
from decode_fish.funcs.predict import window_predict
from decode_fish.funcs.matching import matching
from decode_fish.funcs.merfish_eval import *
from decode_fish.funcs.exp_specific import *
from decode_fish.funcs.routines import *

from omegaconf import open_dict
from hydra import compose, initialize

In [6]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [7]:
initialize(config_path="../config")

hydra.initialize()

In [8]:
cfg = compose(config_name='train.yaml')
cfg = OmegaConf.create(cfg)

## Simple simulations for comparison with ISTDECO (no z, constant BG)

In [12]:
# project and run_name set the directories where the trained models are saved, as well as the structure in wandb.ai
cfg.output.project = 'MERFISH_sim_2D'
cfg.run_name ='sim_2d_2'

cfg.codebook._target_ = 'decode_fish.funcs.exp_specific.get_starfish_codebook'

cfg.genm.exp_type.n_channels = 16
cfg.genm.exp_type.n_genes = 140
cfg.genm.exp_type.n_bits = 4
cfg.genm.exp_type.sample_from_codebook = True
cfg.genm.exp_type.pred_z = False

cfg.data_path.image_sim.image_shape = [1,16,1,48,48]

'''Simulation parameters'''
cfg.genm.PSF.gauss_radii = [1.,1.5,1.5]

cfg.genm.foci.n_foci_avg = 0
    
cfg.genm.prob_generator.low = 0.01
cfg.genm.prob_generator.high = 0.3

cfg.genm.intensity_dist.int_conc = 3.
cfg.genm.intensity_dist.int_rate = 1.
cfg.genm.intensity_dist.int_loc = 1.

cfg.genm.microscope.scale = 300
cfg.genm.emitter_noise.rate_fac = 0.

cfg.genm.pos_noise.pos_noise_z = 0.
cfg.genm.pos_noise.pos_noise_xy = 0.

cfg.sim.bg_estimation.type = 'uniform'
cfg.sim.bg_estimation.uniform.min_val = 99.99
cfg.sim.bg_estimation.uniform.max_val = 100.01

'''Performance tracking'''
cfg.output.wandb_mode = 'online'
cfg.output.log_interval = 500

'''Network architecture'''
cfg.network.f_maps = 256
cfg.network.depth = 2
cfg.network.is_2D = True

cfg.network.inp_scale = 300
cfg.network.inp_offset = 100 

'''Training'''
cfg.training.bs = 8

cfg.training.net.opt.lr = 5e-4
cfg.training.net.bl_loss_scale = 0.0

cfg.training.num_iters = 40000

Save cfg

In [13]:
fname = f'../config/experiment/{cfg.run_name}.yaml'
OmegaConf.save(cfg, fname)
prepend_line(fname, '# @package _global_')
cfg.run_name

'sim_2d_2'

In [16]:
home_dir = os.getenv("HOME")
print(f'bsub -gpu "num=1" -q gpu_rtx -o {home_dir}/logs/ofish.log -e {home_dir}/logs/efish.log {home_dir}/anaconda3/envs/decode_fish_dev2/bin/python {home_dir}/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment={cfg.run_name} run_name=hd2 output.group={cfg.run_name}')

bsub -gpu "num=1" -q gpu_rtx -o /groups/turaga/home/speisera/logs/ofish.log -e /groups/turaga/home/speisera/logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=sim_2d_2 run_name=hd2 output.group=sim_2d_2


In [586]:
img_3d, decode_dl = get_dataloader(cfg)
psf, noise, micro = load_psf_noise_micro(cfg)

Crop size larger than volume in at least one dimension. Crop size changed to (1, 48, 48)
1 volumes


TypeError: cannot unpack non-iterable Microscope object

## Simulations for internal testing of gen. model learning

In [96]:
cfg = compose(config_name='train.yaml', overrides= ["+exp_type=merfish_MOp"])
del(cfg.data_path.image_proc.override)

In [97]:
cfg.output.project = 'MERFISH_sim_int'
cfg.run_name ='sim_int_7_os2'

cfg.data_path.image_path = '/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/data/merfish_sims/sim_int_7_os/images.tiff'

cfg.genm.foci.n_foci_avg = 0

cfg.genm.prob_generator.low = 0.03
cfg.genm.prob_generator.high = 0.1

cfg.output.wandb_mode = 'online'

cfg.network.f_maps = 384
cfg.network.depth = 2
cfg.training.bs = 10

cfg.genm.exp_type.randomize_noise_range = [0.3, 3.]
cfg.genm.noise.theta = 1.
cfg.genm.microscope.ch_facs = None
cfg.genm.microscope.col_shifts_enabled = False

cfg.sim.roi_mask.percentile = 1
cfg.sim.random_crop.crop_sz = 56

cfg.sim.bg_estimation.smoothing.smoothing_filter_size = 9
cfg.sim.bg_estimation.type = 'smoothing'

cfg.genm.intensity_dist.int_conc = 3.
cfg.genm.intensity_dist.int_rate = 1.
cfg.genm.intensity_dist.int_loc = 1.

cfg.training.num_iters = 40000

cfg.genm.microscope.scale = 7000
cfg.genm.phasing = 0.0

cfg.genm.PSF.n_cols = 1
cfg.genm.microscope.ch_cols = None

cfg.data_path.psf_path = '/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/figures/MF_psf_07_20n.tif'
cfg.genm.PSF.gauss_radii = [1.,1.,1.1]
cfg.genm.PSF.psf_extent_zyx = [7,21,21]
# cfg.data_path.psf_path = None

cfg.genm.microscope.norm = 'escort'
cfg.genm.PSF.mode = 'bicubic'

cfg.training.mic.mean_diff = True

cfg.training.mic.par_grads.channel_facs = True
cfg.training.mic.par_grads.channel_shifts = True
cfg.training.mic.par_grads.theta_par = False
cfg.training.mic.par_grads.psf_vol = True
cfg.training.mic.par_grads.color_shifts = False
cfg.training.mic.par_grads.z_facs = False
cfg.training.mic.par_grads.sc_fac = False

cfg.output.log_interval = 600

cfg.genm.emitter_noise.rate_fac = 3.

cfg.training.net.opt.lr = 4e-4

cfg.evaluation.code_stats.enabled = False
cfg.training.schedule = [10000, 2000, 5000, 1000, 5000, 1000, 5000, 1000, 10000]

cfg.training.mic.opt.lr = 0.001

cfg.training.psf.opt.lr = 0.0002
cfg.training.psf.sched.step_size = 2000
cfg.training.psf.l1_reg = 0.0

cfg.training.net.bl_loss_scale = .003

cfg.data_path.model_init  = None
cfg.data_path.micro_init  = None

cfg.training.target_mean = True

cfg.data_path.micro_init  = '/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm//models/fishcod/MERFISH_sim_int/sim_int_7_os2/ae_train/checkpoint/'
cfg.data_path.model_init  = '/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm//models/fishcod/MERFISH_sim_int/sim_int_7_os2/ae_train/checkpoint/'
cfg.network.is_2D = True
cfg.network.pred_z = True

cfg.sim.bg_estimation.shuffle_ch = True
cfg.genm.exp_type.em_noise_inf = True

In [98]:
cfg.training.mic.mean_diff = False

In [99]:
fname = f'../config/experiment/{cfg.run_name}.yaml'
OmegaConf.save(cfg, fname)
prepend_line(fname, '# @package _global_')
cfg.run_name

'sim_int_7_os2'

In [100]:
name = 'sl_true_gen'
home_dir = os.getenv("HOME")
print(f'bsub -gpu "num=1" -q gpu_rtx -o {home_dir}/logs/ofish.log -e {home_dir}/logs/efish.log {home_dir}/anaconda3/envs/decode_fish_dev2/bin/python {home_dir}/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment={cfg.run_name} training.schedule=Null run_name={name} output.group={cfg.run_name} training.num_iters=35000 data_path.psf_path=Null training.mic.enabled=False data_path.model_init=null data_path.micro_init=/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/data/merfish_sims/sim_int_7_os/')

bsub -gpu "num=1" -q gpu_rtx -o /groups/turaga/home/speisera/logs/ofish.log -e /groups/turaga/home/speisera/logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=sim_int_7_os2 training.schedule=Null run_name=sl_true_gen output.group=sim_int_7_os2 training.num_iters=35000 data_path.psf_path=Null training.mic.enabled=False data_path.model_init=null data_path.micro_init=/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/data/merfish_sims/sim_int_7_os/


In [101]:
name = 'ae_train_edgediff'
home_dir = os.getenv("HOME")
print(f'bsub -gpu "num=1" -q gpu_rtx -o {home_dir}/logs/ofish.log -e {home_dir}/logs/efish.log {home_dir}/anaconda3/envs/decode_fish_dev2/bin/python {home_dir}/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment={cfg.run_name} run_name={name} output.group={cfg.run_name}')

bsub -gpu "num=1" -q gpu_rtx -o /groups/turaga/home/speisera/logs/ofish.log -e /groups/turaga/home/speisera/logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=sim_int_7_os2 run_name=ae_train_edgediff output.group=sim_int_7_os2


In [66]:
name = 'ae_train_scf'
home_dir = os.getenv("HOME")
print(f'bsub -gpu "num=1" -q gpu_rtx -o {home_dir}/logs/ofish.log -e {home_dir}/logs/efish.log {home_dir}/anaconda3/envs/decode_fish_dev2/bin/python {home_dir}/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment={cfg.run_name} run_name={name} output.group={cfg.run_name} training.mic.par_grads.sc_fac=True')

bsub -gpu "num=1" -q gpu_rtx -o /groups/turaga/home/speisera/logs/ofish.log -e /groups/turaga/home/speisera/logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=sim_int_7_os2 run_name=ae_train_scf output.group=sim_int_7_os2 training.mic.par_grads.sc_fac=True


In [56]:
name = 'sl_no_learn'
home_dir = os.getenv("HOME")
print(f'bsub -gpu "num=1" -q gpu_rtx -o {home_dir}/logs/ofish.log -e {home_dir}/logs/efish.log {home_dir}/anaconda3/envs/decode_fish_dev2/bin/python {home_dir}/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment={cfg.run_name} run_name={name} output.group={cfg.run_name} training.mic.enabled=False training.schedule=Null')

bsub -gpu "num=1" -q gpu_rtx -o /groups/turaga/home/speisera/logs/ofish.log -e /groups/turaga/home/speisera/logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=sim_int_7_os2 run_name=sl_no_learn output.group=sim_int_7_os2 training.mic.enabled=False training.schedule=Null


In [57]:
name = 'ae_train_no_ch_shifts'
home_dir = os.getenv("HOME")
print(f'bsub -gpu "num=1" -q gpu_rtx -o {home_dir}/logs/ofish.log -e {home_dir}/logs/efish.log {home_dir}/anaconda3/envs/decode_fish_dev2/bin/python {home_dir}/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment={cfg.run_name} run_name={name} output.group={cfg.run_name} training.mic.par_grads.channel_shifts=False')

bsub -gpu "num=1" -q gpu_rtx -o /groups/turaga/home/speisera/logs/ofish.log -e /groups/turaga/home/speisera/logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=sim_int_7_os2 run_name=ae_train_no_ch_shifts output.group=sim_int_7_os2 training.mic.par_grads.channel_shifts=False


In [53]:
name = 'ae_train_no_ch_facs'
home_dir = os.getenv("HOME")
print(f'bsub -gpu "num=1" -q gpu_rtx -o {home_dir}/logs/ofish.log -e {home_dir}/logs/efish.log {home_dir}/anaconda3/envs/decode_fish_dev2/bin/python {home_dir}/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment={cfg.run_name} run_name={name} output.group={cfg.run_name} training.mic.par_grads.channel_facs=False')

bsub -gpu "num=1" -q gpu_rtx -o /groups/turaga/home/speisera/logs/ofish.log -e /groups/turaga/home/speisera/logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=sim_int_7_os2 run_name=ae_train_no_ch_facs output.group=sim_int_7_os2 training.mic.par_grads.channel_facs=False


In [55]:
name = 'ae_train_no_psf'
home_dir = os.getenv("HOME")
print(f'bsub -gpu "num=1" -q gpu_rtx -o {home_dir}/logs/ofish.log -e {home_dir}/logs/efish.log {home_dir}/anaconda3/envs/decode_fish_dev2/bin/python {home_dir}/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment={cfg.run_name} run_name={name} output.group={cfg.run_name} training.mic.par_grads.psf_vol=False')

bsub -gpu "num=1" -q gpu_rtx -o /groups/turaga/home/speisera/logs/ofish.log -e /groups/turaga/home/speisera/logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=sim_int_7_os2 run_name=ae_train_no_psf output.group=sim_int_7_os2 training.mic.par_grads.psf_vol=False


In [570]:
cfg.data_path.psf_path = '/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/figures/chall_ds_psf_onesided_normed.tif'

In [571]:
img_5d, decode_dl = get_dataloader(cfg)
micro = load_psf_noise_micro(cfg)

Crop size larger than volume in at least one dimension. Crop size changed to (1, 56, 56)
10 volumes


/groups/turaga/home/speisera/Dropbox (mackelab)/Artur/WorkDB/deepstorm/decode_fish/decode_fish/engine/psf.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  psf_vol = torch.tensor(psf_vol, dtype=torch.float32).to(device)


In [572]:
cfg.genm.intensity_dist

{'int_conc': 3.0, 'int_rate': 1.0, 'int_loc': 1.0}

In [590]:
from decode_fish.engine.point_process import *
point_process = PointProcessUniform(**cfg.genm.intensity_dist, n_channels=cfg.genm.exp_type.n_channels, sim_z=True, slice_rec=True, codebook=None, int_option=1)
get_simulation_statistics(decode_dl, micro, point_process, int_threshold=0.1, samples=10)

RuntimeError: The size of tensor a (16) must match the size of tensor b (22) at non-singleton dimension 1